# Laboratorio 2
## Detección de SPAM
Preprocesamiento del lenguaje para detección de SPAM en correos electrónicos utilizando ML
> #### Gerardo Méndez 18239, Luis Pedro Cuéllar 18220 

In [1]:
import pandas as pd
import numpy as np
import re

%matplotlib inline

In [59]:
## cargamos los sets de datos
csa_df = pd.read_csv('datasets/completeSpamAssassin.csv')
ess_df = pd.read_csv('datasets/enronSpamSubset.csv')

### Parte 1 - Ingeniería de Características

#### Exploración de datos

In [60]:
## mostramos los datos de csa
csa_df.head()

,Unnamed: 0,Body,Label
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,3,##############################################...,1
4,4,I thought you might like these:\n1) Slim Down ...,1


In [61]:
## mostramos los datos de ess
ess_df.head()

,Unnamed: 0,Unnamed: 0.1,Body,Label
0,2469,2469,Subject: stock promo mover : cwtd\n * * * urge...,1
1,5063,5063,Subject: are you listed in major search engine...,1
2,12564,12564,"Subject: important information thu , 30 jun 20...",1
3,2796,2796,Subject: = ? utf - 8 ? q ? bask your life with...,1
4,1468,1468,"Subject: "" bidstogo "" is places to go , things...",1


In [62]:
## mostramos entradas donde la columna 1 sea diferente a la columna 2
dist = np.where(ess_df.iloc[:, 0] != ess_df.iloc[:, 1])

dist

(array([], dtype=int64),)

In [63]:
## la columna 1 y 2 del segundo dataset es identica, podemos eliminar una
ess_df.drop('Unnamed: 0.1', inplace=True, axis=1)

In [64]:
## unimos los datasets con el mismo numero de columnas
df = csa_df.append(ess_df, ignore_index=True)

In [65]:
df

,Unnamed: 0,Body,Label
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,3,##############################################...,1
4,4,I thought you might like these:\n1) Slim Down ...,1
...,...,...,...
16041,26222,"Subject: monday 22 nd oct\n louise ,\n do you ...",0
16042,18630,Subject: missing bloomberg deals\n stephanie -...,0
16043,18451,Subject: eops salary survey questionnaire\n we...,0
16044,21955,"Subject: q 3 comparison\n hi louise ,\n i have...",0


In [66]:
## rename de columnas para mejor entendimiento
df.rename(columns={"Unnamed: 0": "serial", "Body": "body", "Label": "label"})

,serial,body,label
0,0,\nSave up to 70% on Life Insurance.\nWhy Spend...,1
1,1,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
2,2,1) Fight The Risk of Cancer!\nhttp://www.adcli...,1
3,3,##############################################...,1
4,4,I thought you might like these:\n1) Slim Down ...,1
...,...,...,...
16041,26222,"Subject: monday 22 nd oct\n louise ,\n do you ...",0
16042,18630,Subject: missing bloomberg deals\n stephanie -...,0
16043,18451,Subject: eops salary survey questionnaire\n we...,0
16044,21955,"Subject: q 3 comparison\n hi louise ,\n i have...",0


#### Preprocesamiento

In [ ]:
## TODO: conversiones y modificaciones necesarias en la columna 'body'